### Dynamics of Price
Based on the celebrated APT model and Berra's risk model, I proposed the stock forecasting model as the following expression.

$$r_{i}(t) = \alpha_i(t) + \sum_{k=1}^K \beta_{ik}(t) F_{ik}(t)$$

Note that here we don't assume the excessive return or factor returns to be constant, which is different comparing to APT and Berra model. The traditional way to handle it is doing the time-series calibration (APT) or cross-section regression (Berra). And we would implement in the end of this part.


In [120]:
import pandas as pd
import statsmodels.api as sm
from statsmodels import regression,stats
import itertools
from pykalman import KalmanFilter
idx = pd.IndexSlice

class GeneralizedAPTModel:
    def __init__(self, price_df, equity_df, benchmark_df, factor_list, universe):
        self.price_df = price_df
        self.equity_df = equity_df
        self.benchmark_df = benchmark_df
        self.factor_list = factor_list
        self.universe = universe
        self.valid_universe = None
        self.subset = None
        
        # z-score in universe
        tmp_equity_df = self.calculate_subset_df(self.equity_df,self.universe)
        tmp_equity_df = tmp_equity_df[factor_list].fillna(0) # TODO: fill or drop??
        self.factor_zscore = (tmp_equity_df - tmp_equity_df.groupby(level='date').mean())/tmp_equity_df.groupby(level='date').std()
        
        # valide_universe
        self.valid_universe = self.calculate_valid_universe()


    def calculate_valid_universe(self):
        valid_universe = set(self.universe)

        for date,group in self.factor_zscore.groupby(level=0):
            size = group.shape[0]
            new_set = set(group.loc[idx[:,valid_universe],:].index.get_level_values(1).values)
            if(new_set<valid_universe):
                valid_universe = new_set
        # self.valid_universe = valid_universe        
        return valid_universe

    def calculate_subset_df(self,df,subset=None):
        if(subset!=None):
            return df.loc[idx[:,subset],:]
        else:
            return df.loc[idx[:,self.subset],:]
        
    def time_series_regression(self):
        ts_factor_zscore = sm.add_constant(self.factor_zscore)
        Y = [self.equity_df.xs(asset,level=1)['return'] for asset in self.valid_universe]
        X = [ts_factor_zscore.xs(asset,level=1)[factor_list+['const']] for asset in self.valid_universe]
        reg_results = [regression.linear_model.OLS(y,x).fit().params for y,x in zip(Y,X) if not(x.empty or y.empty)]
        indices = [asset for y, x, asset in zip(Y, X, self.valid_universe) if not(x.empty or y.empty)]
        ts_result_df = pd.DataFrame(reg_results, index=indices)
        return ts_result_df

    def cross_section_regression(self, ndays=22, latest=True):
        valid_factor_zscore = self.factor_zscore.loc[idx[:,self.valid_universe],:]
        if(latest):
            dates = valid_factor_zscore.index.get_level_values(level=0).unique()[-ndays:]
        else:
            dates = valid_factor_zscore.index.get_level_values(level=0).unique()[:ndays]
        valid_factor_zscore = valid_factor_zscore.loc[idx[dates,:],:]
        
        result_list = []
        date_list = []
        for date,group in valid_factor_zscore.groupby(level=0):
#             print(date)
#             print(group)
            X = sm.add_constant(group.loc[:,self.factor_list])
            y = list(equity_df.loc[(date,list(self.valid_universe)),'return'])          
#             print(np.shape(X))
#             print(np.shape(y))
            results = sm.regression.linear_model.OLS(y,X).fit()
#             print(results.summary())
            result_list.append(results.params)
            date_list.append(date)

        cs_result_df = pd.DataFrame(result_list,index=date_list)
        return cs_result_df
    
    def kalman_filter_calibration(self):
        pass
        

In [2]:
from rqdata_utils import *
import pandas as pd
import numpy as np
import scipy as sp
import alphalens as al
from pykalman import KalmanFilter


price_df,instrument_df,equity_df = get_price_instrument_equity("cn_stock_price_2012_2018.csv","cn_instrument_info_2012_2018.csv","cn_equity_daily_2012_2018.csv","sectorCode")
healthcare_universe = instrument_df.index[instrument_df.sectorCode=='HealthCare'].values
benchmark_df = benchmark_reader("cn_SH_healthcare_index_2012_2018.csv")
factor_list = ['market_cap', 'pb_ratio', 'ps_ratio']
universe_list = instrument_df.index[instrument_df.sectorCode=='HealthCare'].values

In [121]:
myModel = GeneralizedAPTModel(price_df, equity_df, benchmark_df, factor_list, universe_list)

/home/jerryxyx/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:37: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [122]:
myModel.cross_section_regression()

,const,market_cap,pb_ratio,ps_ratio
2018-03-27,0.020801,-0.005600,-0.000886,-0.000948
2018-03-28,-0.006505,-0.004983,-0.000603,0.001667
2018-03-29,0.002383,-0.003000,-0.000925,0.001257
2018-03-30,0.014809,0.004416,0.000530,0.001303
2018-04-02,-0.000119,-0.003920,-0.000729,0.000986
2018-04-03,-0.000223,-0.000844,0.003152,0.003903
2018-04-04,0.012473,-0.002332,-0.003176,-0.004564
2018-04-09,0.001731,-0.001213,-0.003338,0.007950
2018-04-10,0.004090,0.003486,-0.003047,0.009011
2018-04-11,-0.002662,-0.001404,0.002325,0.004528


In [123]:
myModel.time_series_regression()

,market_cap,pb_ratio,ps_ratio,const
000952.XSHE,-0.009326,0.001813,0.019116,-0.001008
000989.XSHE,-0.003066,-0.001685,0.000733,0.000091
600285.XSHG,0.009858,-0.000785,0.010601,0.009665
600645.XSHG,0.004443,0.000926,-0.000456,0.000012
000597.XSHE,0.011020,-0.001664,0.002069,0.008480
600161.XSHG,0.021259,-0.002718,-0.000648,-0.001337
000739.XSHE,0.009936,0.002415,0.004234,0.008364
600771.XSHG,0.003142,0.000345,-0.000197,0.002099
000908.XSHE,0.004476,0.000006,0.001237,0.002244
000790.XSHE,0.005850,0.001411,0.008723,0.003985
